In [20]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.window import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
spark=create_spark_session()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
input_data = 's3a://udacity-dend/'
output_data = 's3a://spark-and-data-lake/output/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Songs Data

In [4]:
song_data = input_data + 'song_data/*/*/*/'
song_df = spark.read.json(song_data)
song_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[artist_id: string, artist_latitude: double, artist_location: string, artist_longitude: double, artist_name: string, duration: double, num_songs: bigint, song_id: string, title: string, year: bigint]

## Songs Table

In [5]:
songs_table = song_df.select(["song_id", "title", "artist_id", "year", "duration"]).distinct()
songs_table.show(5)
songs_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOFXVGR12A67AE0974|I'm A Star (Expli...|ARWVM0Q1187B9A2AFD|2005|237.19138|
|SOVVCEK12A6D4F6373|Everything You Al...|ARRL7WS1187FB576F1|   0|165.98159|
|SOGJQWG12A81C20AE7|IX E Major [24 Pr...|ARC5EHQ1187FB4834F|   0|227.73506|
|SOAYZVD12A6D4F8145|Section  53_ Row ...|AR6PETN1187B998AA2|   0|312.16281|
|SOYVBGZ12A6D4F92A8|Piano Sonata No. ...|ARLRWBW1242077EB29|   0|221.70077|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)

In [6]:
songs_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

## Artists Table

In [7]:
artists_table = song_df.select(["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]).distinct()
artists_table.show(5)
artists_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARAILTA11F4C840A06|Endellion String ...|                    |           null|            null|
|ARBB8QH1187FB54138|Paul Hindemith/Ph...|                    |           null|            null|
|AROU8WM1187B9B4620|   Milton Nascimento|Rio de Janeiro, B...|      -22.97673|       -43.19508|
|ARL1MEI1187B9AD98B|Streetlight Manif...|New Brunswick, Ne...|           null|            null|
|AR1L2NA119B86684B4|Dr. Hook And The ...|Union City, New J...|       40.77387|       -74.02469|
+------------------+--------------------+--------------------+---------------+----------------+
only showing top 5 rows

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: 

In [8]:
artists_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10025

# Log Data

In [11]:
log_data = input_data + "log_data/*/*/"
log_df = spark.read.json(log_data)
log_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: double, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [12]:
log_df =log_df.where('page="NextSong"')
log_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: double, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

## Users Table

In [13]:
users_table = log_df.select(["userId", "firstName", "lastName", "gender", "level"]).distinct()
users_table.show(5)
users_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    80|    Tegan|  Levine|     F| paid|
|    16|    Rylan|  George|     M| paid|
|    48|   Marina|  Sutton|     F| free|
|    83|  Stefany|   White|     F| free|
|     8|   Kaylee| Summers|     F| free|
+------+---------+--------+------+-----+
only showing top 5 rows

root
 |-- userId: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

In [14]:
users_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

104

## Time Table

In [15]:
from pyspark.sql.types import TimestampType
get_timestamp = udf(lambda x: datetime.fromtimestamp((x / 1000)), TimestampType())
log_df = log_df.withColumn("timestamp", get_timestamp(col("ts")))
log_df.select('timestamp').show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           timestamp|
+--------------------+
|2018-11-15 00:30:...|
|2018-11-15 00:41:...|
+--------------------+
only showing top 2 rows

In [16]:
time_table = log_df.select(
    col('timestamp').alias('start_time'),
    hour('timestamp').alias('hour'),
    dayofmonth('timestamp').alias('day'),
    weekofyear('timestamp').alias('week'),
    month('timestamp').alias('month'),
    year('timestamp').alias('year'),
    date_format(col("timestamp"), "E").alias("weekday")
)
time_table.show(5)
time_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:...|   0| 15|  46|   11|2018|    Thu|
|2018-11-15 00:41:...|   0| 15|  46|   11|2018|    Thu|
|2018-11-15 00:45:...|   0| 15|  46|   11|2018|    Thu|
|2018-11-15 03:44:...|   3| 15|  46|   11|2018|    Thu|
|2018-11-15 05:48:...|   5| 15|  46|   11|2018|    Thu|
+--------------------+----+---+----+-----+----+-------+
only showing top 5 rows

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: string (nullable = true)

In [17]:
time_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6820

## Songplays Table

In [21]:
song_df.createOrReplaceTempView("song_data")
log_df.createOrReplaceTempView("log_data")
    
songplays_table = spark.sql("""
                            SELECT monotonically_increasing_id() as songplay_id,
                                timestamp as start_time,
                                userId as user_id,
                                level,
                                song_id,
                                artist_id,
                                sessionId as session_id,
                                location,
                                userAgent as user_agent,
                                year(timestamp) as year, 
                                month(timestamp) as month
                            FROM log_data log
                            JOIN song_data song
                            ON (log.song = song.title
                            AND log.length = song.duration
                            AND log.artist = song.artist_name)
                            """)
songplays_table.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|songplay_id|          start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|year|month|
+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|          0|2018-11-15 07:31:...|     49| paid|SOCUITT12AB0187A32|ARKS2FE1187B99325D|       606|San Francisco-Oak...|Mozilla/5.0 (Wind...|2018|   11|
|          1|2018-11-15 10:44:...|     80| paid|SOSDZFY12A8C143718|AR748W61187B9B6AB8|       611|Portland-South Po...|"Mozilla/5.0 (Mac...|2018|   11|
+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
only showing top 2 rows

In [22]:
songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- songplay_id: long (nullable = false)
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

In [23]:
songplays_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

319

## Write Parquet

In [24]:
users_table.write.parquet(output_data + 'users/', mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
songplays_table.write.parquet(output_data + 'songplays/', mode = "overwrite", partitionBy = ['year','month'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
time_table.write.parquet(output_data + 'time/', mode = "overwrite", partitionBy = ["year", "month"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
artists_table.write.parquet(output_data + 'artists/', mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
songs_table.write.parquet(output_data + 'songs_without_partition/', mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
songs_table.write.parquet(output_data + 'songs/', mode = "overwrite", partitionBy = ["year", "artist_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…